## Property Price Register

### Exploratory Spatial Data Analysis

In [1]:
# import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

C:\Users\callm\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('tidyppr.csv', low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408319 entries, 0 to 408318
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 408319 non-null  int64  
 1   year                       408319 non-null  int64  
 2   Date                       408319 non-null  object 
 3   Postal Code                77725 non-null   object 
 4   County                     408319 non-null  object 
 5   Price                      408319 non-null  float64
 6   Not Full Market Price      408319 non-null  object 
 7   VAT Exclusive              408319 non-null  object 
 8   Description of Property    408319 non-null  object 
 9   Property Size Description  45140 non-null   object 
 10  formatted_address          393884 non-null  object 
 11  accuracy                   393888 non-null  object 
 12  latitude                   393888 non-null  float64
 13  longitude                  39

### Convert Data Types

In [4]:
from datetime import datetime

In [5]:
df['Date'] =  pd.to_datetime(df['Date'])

In [6]:
df["Price"] = df['Price'].astype('int64')

In [7]:
df = df.replace({'VAT Exclusive': {'Yes': True, 'No': False}})

In [8]:
df = df.replace({'Not Full Market Price': {'Yes': True, 'No': False}})

In [9]:
df["VAT Exclusive"] = df['VAT Exclusive'].astype('boolean')

In [10]:
df["Not Full Market Price"] = df['Not Full Market Price'].astype('boolean')

1. Based on the information above we can see that 393,888 addresses were geocoded out of a possible 408,319.
2. We can also see that some of the coloumns are missing a lot of values.
2. Below we will remove any columns with more than 40% missing values.

In [11]:
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.4]]
del df2['Unnamed: 0']
print("List of dropped columns:", end=" ")
for c in df.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')
df = df2


List of dropped columns: Unnamed: 0, Postal Code, Property Size Description, 



As we can see from above the columns Unnamed: 0, Postal Code and Property Size Description have been dropped. The reason for this is because data is entered into the Property Price Register manually and does not force the user to complete all fields.

### Cleaning data
1. If a property is VAT Exclusive then the property is new which means we can drop the Description of the property column.
2. As our data is geocoded we no longer need the postcodes so we will also drop it.
3. Accuracy is google response to ensure our data is geocoded correctly at this point we no longer need it so we will drop that too.

In [12]:
df = df.drop(['Description of Property','postcode','accuracy','type'], axis=1)
df

,year,Date,County,Price,Not Full Market Price,VAT Exclusive,formatted_address,latitude,longitude
0,2012,2012-09-18,Cork,118000,False,False,"Saint Martins, Kilbeg, Ightermurragh, Ladysbri...",51.898721,-8.027057
1,2012,2012-01-16,Tipperary,210000,False,False,"The Stone House, Ballyglass Upper, Co. Tippera...",52.438674,-8.171166
2,2012,2012-01-23,Kildare,220000,False,False,"Avondale, Morristown Rd, Moorfield, Newbridge,...",53.178275,-6.815277
3,2012,2012-12-12,Dublin,496089,False,False,"61 Dublin Rd, Miltonsfields, Swords, Co. Dubli...",53.451025,-6.225203
4,2012,2012-01-25,Cork,240000,False,False,"33 Glasheen Rd, Liam Lynch Park, Glasheen, Cor...",51.884846,-8.505120
...,...,...,...,...,...,...,...,...,...
408314,2020,2020-05-22,Tipperary,70000,False,False,"2 Main St, Townparks, Roscrea, Co. Tipperary, ...",52.956481,-7.799102
408315,2020,2020-05-22,Limerick,250000,False,False,"Cloonoughter, Co. Limerick, Ireland",52.568506,-9.246203
408316,2020,2020-05-22,Cork,500000,False,False,"Highgrove, Johnstown, Annmount, Co. Cork, Ireland",51.912757,-8.331703
408317,2020,2020-05-22,Meath,99000,False,False,"Ballinlough, Lough Bawn, Co. Meath, Ireland",53.693480,-7.173900


### Geopandas

Now we will use geopandas to map our small area shape file with our PPR data.

In [13]:
import geopandas

In [14]:
from shapely.geometry import Point, Polygon

In [15]:
small_areas = geopandas.read_file('irl.shp')

In [16]:
small_areas

,OBJECTID,GUID,NUTS1,NUTS1NAME,NUTS2,NUTS2NAME,NUTS3,NUTS3NAME,COUNTY,COUNTYNAME,...,ESRI_OID,Shape__Are,Shape__Len,TotalPop,Housing St,Occupied,Unoccupied,Temporaril,VacantPerc,geometry
0,1,4c07d11d-f71d-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE05,Southern,IE053,South-West,CK,Cork County,...,1,2.436209e+07,25179.376645,154,75,61,2.0,2.0,6.3,"POLYGON ((-9.09329 51.79897, -9.09309 51.79897..."
1,2,4c07d11d-f71e-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE05,Southern,IE053,South-West,CK,Cork County,...,2,1.215051e+07,23595.409065,283,108,94,2.0,5.0,18.5,"POLYGON ((-8.98764 51.72759, -8.98742 51.72763..."
2,3,4c07d11d-f71f-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE05,Southern,IE053,South-West,CK,Cork County,...,3,9.331929e+06,18078.254302,228,92,73,1.0,1.0,16.5,"POLYGON ((-8.93013 51.72561, -8.93007 51.72566..."
3,4,4c07d11d-f720-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE05,Southern,IE053,South-West,CK,Cork County,...,4,1.134573e+07,22123.902506,180,118,78,20.0,1.0,2.1,"POLYGON ((-8.92680 51.73688, -8.92685 51.73711..."
4,5,4c07d11d-f564-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE04,Northern and Western,IE042,West,GY,Galway County,...,5,8.135992e+07,71992.829074,194,121,68,27.0,3.0,9.1,"POLYGON ((-9.54482 53.42068, -9.54485 53.42069..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18636,18637,4c07d11e-3b99-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE06,Eastern and Midland,IE061,Dublin,DR,DÃºn Laoghaire-Rathdown,...,18637,1.493571e+04,497.940601,349,97,97,NaN,NaN,3.9,"POLYGON ((-6.14090 53.26959, -6.14095 53.26953..."
18637,18638,4c07d11e-3b9a-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE06,Eastern and Midland,IE061,Dublin,DR,DÃºn Laoghaire-Rathdown,...,18638,1.004670e+04,462.357975,148,107,75,NaN,7.0,6.1,"POLYGON ((-6.24302 53.27999, -6.24303 53.28028..."
18638,18639,4c07d11e-3b9b-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE06,Eastern and Midland,IE061,Dublin,DR,DÃºn Laoghaire-Rathdown,...,18639,1.301185e+04,492.908907,134,85,77,NaN,1.0,3.5,"POLYGON ((-6.21417 53.27828, -6.21514 53.27721..."
18639,18640,4c07d11e-3baa-851d-e053-ca3ca8c0ca7f,IE,Ireland,IE06,Eastern and Midland,IE061,Dublin,FL,Fingal,...,18640,2.280690e+04,625.759061,138,77,57,NaN,8.0,10.4,"POLYGON ((-6.20373 53.61362, -6.20370 53.61351..."


In [17]:
small_areas = small_areas[['OBJECTID', 'geometry']]

In [18]:
# Converting dataframe to geodataframe
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
gdf

,year,Date,County,Price,Not Full Market Price,VAT Exclusive,formatted_address,latitude,longitude,geometry
0,2012,2012-09-18,Cork,118000,False,False,"Saint Martins, Kilbeg, Ightermurragh, Ladysbri...",51.898721,-8.027057,POINT (-8.02706 51.89872)
1,2012,2012-01-16,Tipperary,210000,False,False,"The Stone House, Ballyglass Upper, Co. Tippera...",52.438674,-8.171166,POINT (-8.17117 52.43867)
2,2012,2012-01-23,Kildare,220000,False,False,"Avondale, Morristown Rd, Moorfield, Newbridge,...",53.178275,-6.815277,POINT (-6.81528 53.17828)
3,2012,2012-12-12,Dublin,496089,False,False,"61 Dublin Rd, Miltonsfields, Swords, Co. Dubli...",53.451025,-6.225203,POINT (-6.22520 53.45102)
4,2012,2012-01-25,Cork,240000,False,False,"33 Glasheen Rd, Liam Lynch Park, Glasheen, Cor...",51.884846,-8.505120,POINT (-8.50512 51.88485)
...,...,...,...,...,...,...,...,...,...,...
408314,2020,2020-05-22,Tipperary,70000,False,False,"2 Main St, Townparks, Roscrea, Co. Tipperary, ...",52.956481,-7.799102,POINT (-7.79910 52.95648)
408315,2020,2020-05-22,Limerick,250000,False,False,"Cloonoughter, Co. Limerick, Ireland",52.568506,-9.246203,POINT (-9.24620 52.56851)
408316,2020,2020-05-22,Cork,500000,False,False,"Highgrove, Johnstown, Annmount, Co. Cork, Ireland",51.912757,-8.331703,POINT (-8.33170 51.91276)
408317,2020,2020-05-22,Meath,99000,False,False,"Ballinlough, Lough Bawn, Co. Meath, Ireland",53.693480,-7.173900,POINT (-7.17390 53.69348)


In [19]:
# Now we have created a geometric point for each address we can drop the latitude and longitude along with strings.
geo = gdf.drop(['County','year','formatted_address','latitude','longitude'], axis=1)

## Coordinate Reference System

In [20]:
geo.crs = {'init' :'epsg:4326'}

In [21]:
geo.crs

{'init': 'epsg:4326'}

In [22]:
small_areas.crs

{'init': 'epsg:4326'}

In [23]:
geo

,Date,Price,Not Full Market Price,VAT Exclusive,geometry
0,2012-09-18,118000,False,False,POINT (-8.02706 51.89872)
1,2012-01-16,210000,False,False,POINT (-8.17117 52.43867)
2,2012-01-23,220000,False,False,POINT (-6.81528 53.17828)
3,2012-12-12,496089,False,False,POINT (-6.22520 53.45102)
4,2012-01-25,240000,False,False,POINT (-8.50512 51.88485)
...,...,...,...,...,...
408314,2020-05-22,70000,False,False,POINT (-7.79910 52.95648)
408315,2020-05-22,250000,False,False,POINT (-9.24620 52.56851)
408316,2020-05-22,500000,False,False,POINT (-8.33170 51.91276)
408317,2020-05-22,99000,False,False,POINT (-7.17390 53.69348)


In [24]:
geo = geopandas.sjoin(geo, small_areas, how="inner", op='within')
geo

C:\Users\callm\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,Date,Price,Not Full Market Price,VAT Exclusive,geometry,index_right,OBJECTID
0,2012-09-18,118000,False,False,POINT (-8.02706 51.89872),3525,3526
19642,2012-06-07,220000,False,False,POINT (-8.04333 51.89761),3525,3526
24083,2012-05-04,205000,False,False,POINT (-8.04333 51.89761),3525,3526
34998,2013-05-23,220000,False,False,POINT (-8.04308 51.89556),3525,3526
51394,2013-01-23,165000,False,False,POINT (-8.04286 51.89715),3525,3526
...,...,...,...,...,...,...,...
406480,2020-04-24,33000,False,False,POINT (-7.94840 53.55361),1911,1912
406606,2020-04-27,300000,True,False,POINT (-7.35693 52.94567),4594,4595
406685,2020-04-28,37500,False,False,POINT (-7.58791 53.79392),3197,3198
407684,2020-11-05,177400,False,False,POINT (-8.76558 53.83057),3639,3640


In [25]:
geo.to_csv('geoppr.csv')

## Save to csv

The above function took several minutes to complete so at this point we will save the results.